# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [148]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [149]:
# Save path to SFH_data set in a variable
city_data_file = "../WeatherPY/output_data/cities.csv"
city_data_df = pd.read_csv(city_data_file)
city_data_df.describe()

,Unnamed: 0,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,566.0000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,5.660000e+02
mean,282.5000,21.394806,19.942297,60.933127,68.418728,51.975265,8.120230,1.587770e+09
std,163.5344,32.889363,90.044692,19.712075,22.196379,36.878553,5.784607,1.408598e+02
min,0.0000,-54.800000,-175.200000,-2.200000,6.000000,0.000000,0.340000,1.587769e+09
25%,141.2500,-4.040000,-60.557500,46.400000,56.000000,14.250000,4.000000,1.587770e+09
50%,282.5000,24.110000,24.550000,63.000000,73.000000,55.000000,7.090000,1.587770e+09
75%,423.7500,49.522500,102.852500,77.652500,85.000000,90.000000,11.247500,1.587770e+09
max,565.0000,77.480000,179.320000,104.000000,100.000000,100.000000,55.250000,1.587770e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [164]:
# generate some (latitude, longitude) pairs
locations = city_data_df[["Lat", "Lng"]]
center_map = ["0", "0"]

NameError: name 'Lat' is not defined

In [151]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure(map_type='ROADMAP')

heatmap_layer = gmaps.heatmap_layer(
    city_data_df[["Lat", "Lng"]], weights=city_data_df["Humidity"],
    max_intensity=100, point_radius=7.5)
fig = gmaps.figure(center=center_map, zoom_level=1.9, layout = {'width' : '100%', 'height' : '675px'})
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='675px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [248]:
#Narrow down the DataFrame to find your ideal weather condition. 
# A max temperature lower than 75 degrees but higher than 55.
#Humidity < 50%
#Wind speed less than 7.5 mph.
prime_weather_df = city_data_df[city_data_df["Max Temp"] <75]
prime_weather_df = prime_weather_df[prime_weather_df["Max Temp"]>55]
prime_weather_df = prime_weather_df[prime_weather_df["Humidity"]<40]
prime_weather_df = prime_weather_df[prime_weather_df["Wind Speed"]<7]
prime_weather_df.dropna()
#change lat and long to strings
#prime_weather["Lat"] = prime_weather["Lat"].astype(str)
#prime_weather["Lng"] = prime_weather["Lng"].astype(str)
#added location column 
#prime_weather["Location"] = prime_weather["Lat"].str.cat(prime_weather["Lng"],sep=", ")
prime_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Location
77,77,Shingū,33.73,135.98,57.18,35,0,4.21,JP,1587769853,"33.73, 135.98"
162,162,Nampa,43.54,-116.56,66.00,31,90,3.36,US,1587769886,"43.54, -116.56"
164,164,Kutum,14.2,24.67,67.37,14,53,6.89,SD,1587769886,"14.2, 24.67"
203,203,Atar,20.52,-13.05,70.88,21,13,6.85,MR,1587769902,"20.52, -13.05"
264,264,Sehithwa,-20.47,22.72,67.24,36,33,2.84,BW,1587769931,"-20.47, 22.72"
307,307,Aksu,41.12,80.26,59.34,36,48,3.20,CN,1587769950,"41.12, 80.26"
358,358,Tsumeb,-19.23,17.72,58.60,34,2,2.59,NaN,1587769976,"-19.23, 17.72"
385,385,Shahreza,32.03,51.88,57.81,21,3,6.02,IR,1587769991,"32.03, 51.88"
464,464,Hermon,44.81,-68.91,55.40,16,90,5.82,US,1587770028,"44.81, -68.91"
479,479,Yinchuan,38.47,106.27,57.09,13,0,6.60,CN,1587770035,"38.47, 106.27"


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = prime_weather

In [267]:

# PARAMS Dictionary updates each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
for index, city in prime_weather.iterrows():
    # get lat, lng from df
    lat = city["Lat"]
    lng = city["Lng"]
# Change location for each iteration while leaving original params in place
    params["location"] = (f"{lat},{lng}")
    
    # Create url and make API request
    print(f"Retrieving Results for Index {index}: {prime_weather['City']}.")
    response = requests.get(base_url, params=params).json()
     
    # Extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        prime_weather.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("EXCEPTION: City name is not found. Skip to next city...")
        
    print("-------------------------------------------------------")

Retrieving Results for Index 77: 77          Shingū
162          Nampa
164          Kutum
203           Atar
264       Sehithwa
307           Aksu
358         Tsumeb
385       Shahreza
464         Hermon
479       Yinchuan
504       Mingshui
509          Gimbi
521    Fort Nelson
Name: City, dtype: object.
Closest hotel is 新宮ユーアイホテル.
-------------------------------------------------------
Retrieving Results for Index 162: 77          Shingū
162          Nampa
164          Kutum
203           Atar
264       Sehithwa
307           Aksu
358         Tsumeb
385       Shahreza
464         Hermon
479       Yinchuan
504       Mingshui
509          Gimbi
521    Fort Nelson
Name: City, dtype: object.
Closest hotel is Nampa Downtown.
-------------------------------------------------------
Retrieving Results for Index 164: 77          Shingū
162          Nampa
164          Kutum
203           Atar
264       Sehithwa
307           Aksu
358         Tsumeb
385       Shahreza
464         Hermon
479    

In [269]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in prime_weather.iterrows()]
locations = prime_weather[["Lat", "Lng"]]

In [270]:
# Add marker layer ontop of heat map
target_corrdinates
markers = gmaps.marker_layer(target_corrdinates)
fig.add_layer(markers)
# Display figure
fig

TypeError: unsupported operand type(s) for -: 'str' and 'str'